#### Glosario comandos de linux

In [ ]:
$ ls : Lista archivos y directorios.
$ pwd : Muestra el path actual.
$ mkdir : Crea nuevos directorios.
$ rm : Elimina archivos y directorios.
$ mv : Mueve o renombra archivos y directorios.
$ cp : Copia archivos y directorios.

$ stat : Muestra el estado de un archivo o sistema de archivos (en un enlace simbolico info del fichero original).
$ fstat : Muestra el estado de un sistema de ficheros.
$ lstat : Muestra el estado del propio enlace.
$ touch : Actualiza los tiempos de acceso y modificación de un fichero (si no existe lo crea).

$ chmod : Cambia los permisos de un archivo o directorio.
$ chown : Cambia el propietario y grupo de un archivo o directorio.
$ chgrp : Cambia el grupo asociado a un archivo o directorio.

$ ln : Crea enlaces físicos o simbólicos entre archivos y directorios.
$ readlink : Leer el contenido de un enlace simbolico.
$ file : Muestra el tipo de un archivo.
$ file $(ls) : utiliza lo devuelto por ls como argumentos para file.

#### Funciones de manejo de ficheros en C (equivalentes a comandos de Linux)

In [ ]:

int link (const chat *old, const char *new) // equivalente a $ ln
int symlink(const char *old, const char *new) // equivalente a $ ln -s
int readlink(const char *path, char *buf, size_t)

// MIRAR DIAPOSITIVA 18 TEMA 2 PARA INFO DE LAS FLAGS.
// path: donde esta el fichero, flags: como abrir el fichero, mode: 
// Devuelve un descriptor de fichero o -1 si hay error 
int open(const char *path, int flags) // Abre un fichero.
int open(const char *path, int flags, mode_t mode) // Abre un fichero con un modo (necesario para algunos flags).

// Devuelve: el numero de bytes que ha leido; 0 si ha llegado al final; -1 si hay error.
ssize_t read(int fd, void *buf, size_t count) // Lee count bytes del descriptor fd (fichero) y los guarda en buf (path); count = cantidad de datos a leer.
ssize_t write(int fd, const void *buf, size_t count) // Escribe count bytes del buffer buf en el fichero fd. Devuelve el numero de bytes escritos o -1 si hay error.
off_t lseek(int fd, off_t offset, int whence) // Cambia la posicion del cursor de lectura/escritura del fichero fd. Devuelve la nueva posicion o -1 si hay error.
int close(int fd) // Cierra el fichero fd. Devuelve 0 si todo va bien o -1 si hay error.
/*whence puede ser:
    SEEK_SET: desde el principio del fichero.
    SEEK_CUR: desde la posicion actual.
    SEEK_END: desde el final del fichero.
*/

int fsync(int fd) // Fuerza la sincronizacion de las cache.

// name: path completo
DIR *opendir(const chat *name); // Abre un directorio. Devuelve puntero al directorio.
int closedir(d);

CUIDADO!! NO MEZCLAR LLAMADAS AL SISTEMA DE BAJO NIVEL DE C, CON USO DE LIBRERIAS DE C++.

#### Ejercicio 4 de la hoja

In [ ]:
#include <sys/types.h>
#include <stdio.h>

int main(int argc, char *argv[]) {
    struct stat statbuf;

    int rc = lstat(argv[1], &statbuf);
    if (rc == -1) {
        perror("stat()");
        return 1; // devolver numero distinto de 0 si hay error
    }

    // Utilizacion de las macros para determinar el tipo de archivo
    if (S_DIR(statbuf.st_mode)) { // Pasas el atributo st_mode de la estructura stat
        printf("%s/\n", argv[1]); // Ponemos / para indicar que es un directorio.
    } 
    else if (S_ISREG(statbuf.st_mode)) {
        if (statbuf.st_mode & (S_IXUSR | S_XGRP | S_XOTH)) { // Usamos las mascaras de bits para ver si hay permiso de ejecucion **
            printf("%s*\n", argv[1]); // Ponemos * para indicar que es un ejecutable.
        } 
        else {
            printf("%s-\n", argv[1]); // Ponemos - para indicar que es un fichero regular.
        }
    }

    return 0; // devolver 0 si no hay error
}

#### Ejercicio 8 de la hoja


In [ ]:
# comando DD (1)
> dd if=/dev/urandom of = ./rand bs=512 bc=4 # crea un fichero y contenido random

In [ ]:
#include <fcntl.h>
#include <unistd.h>
#include <stddio.h>
#include <stdlib.h>

int main(int argc, char *argv[]){
    // APERTURA DE FICHEROS.

    int in_fd, out_fd; // Descriptores con los que vamos a trabajar.

    in_fd = open(argv[1], O_RDONLY); // Abrimos el fichero de entrada en modo solo lectura.

    if (in_fd == -1){
        perror("open() in");
        return 1;
    }
     // Abrimos el fichero de salida en modo escrituraONLY, crear si no existe, truncar el contenido.
    out_fd = open(argv[2], O_WRONLI | O_CREAT | O_TRUNC, 0444); // Ponemos en octal los modos necesarios para las flags. (Que empiece por 0 indica que es un numero octal).

    if (in_fd == -1){
        perror("open() out");
        return 1;
    }

    // Cerramos los descriptores.
    close(out_fd); 
    close(in_fd);

    // LECTURA.
    int bs = atoi(argv[3]); // Tamaño del bloque a leer.
    int count = atoi(argv[4]); // Cantidad de bloques a leer.

    char buffer[8192]; // Definimos un buffer de 8192 bytes (que sea char es importante).

    int total_bytes = 0;

    // Bucle para leer un bloque de datos.
    // Leemos por bloques por si no se puede tener toda la informacion.
    while(){
        // buffer + total_bytes: Avanza el puntero hasta donde ya ha leido.
        // bs - total_bytes: lee lo que le falta para completar el bloque.
        int rc = read(in_fd, buffer + total_bytes, bs - total_bytes);

        if (rc == 0){ // si ya se ha terminado la lectura sale del bucle
            break;
        }

        total_bytes += rc;
    }

    if (bytes == 0){

    }else if(bytes <= bs) {// El tamaño leido es menor que el de bloque.
        // Seguimos leyendo no desde el tamaño de bloque, si no desde la cantidad que se ha podido leer, de lo contrario se pierden datos.

    }

    // TODO: ESCRITURA.
        
}

#### Ejercicio 9 de la hoja

In [ ]:
#include <sys/types.h>
#include <stdio.h>

int main(int argc, char *argv[]) {
    Dir * d = opendir(argv[1]);

    struct dirent * dent;

    while ((dent = readdir(d)) != NULL) { // Recorremos el directorio hasta que readdir devuelva NULL (fin del directorio).
        char path[8192]; // Path a un archivo. Usar un define para no poner un numero a ojo.
        snprintf(path,8192,"%s/%s", argv[1], dent->d.d_name) // "%s/%s": <lo que me pasan> "/" <lo que he obtenido>
        printf("---> %s\n", dent->d_name);
        
        closedir(d)


        // Esto es feo, pero es para no complicar el ejemplo.
        struct stat statbuf;

        int rc = lstat(path, &statbuf);
        if (rc == -1) {
            perror("stat()");
            return 1; // devolver numero distinto de 0 si hay error
        }

        // Utilizacion de las macros para determinar el tipo de archivo
        if (S_DIR(statbuf.st_mode)) { // Pasas el atributo st_mode de la estructura stat
            printf("%s/\n", path); // Ponemos / para indicar que es un directorio.
        } 
        else if (S_ISREG(statbuf.st_mode)) {
            if (statbuf.st_mode & (S_IXUSR | S_XGRP | S_XOTH)) { // Usamos las mascaras de bits para ver si hay permiso de ejecucion **
                printf("%s*\n", path); // Ponemos * para indicar que es un ejecutable.
            } 
            else {
                printf("%s-\n", path); // Ponemos - para indicar que es un fichero regular.
            }
        }
    }
    return 0; // devolver 0 si no hay error
}